# Titanic Kaggle competition 


## Objective 
---------
    Predicting survival passengers 

## Data
---------
    <b>Description</b> : The data has been split into two groups 
    - training set (train.csv) 
    - test set (test.csv)
    
    <b>shape </b> : 10 columns 
        survival	Survival	0 = No, 1 = Yes
        pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
        sex	Sex	
        Age	Age in years	
        sibsp	number of siblings / spouses aboard the Titanic	
        parch	number of parents / children aboard the Titanic	
        ticket	Ticket number	
        fare	Passenger fare	
        cabin	Cabin number	
        embarked	Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
    
## Model 
-----------
    What model I used 
    
    
## Results 
------------
    Results

## Import modules 

In [30]:
import numpy as np 
import pandas as pd 

## Import data 

In [31]:
data = pd.read_csv('Data/train.csv', sep=',')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
test_data = pd.read_csv('Data/test.csv', sep=',')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data exploration 

In [37]:
print('Number of passengers :', len(data), ' Missing values (total) : ', data.isnull().sum().sum())

Number of passengers : 891  Missing values (total) :  866


In [39]:
data[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch']].describe()

,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594
std,0.486592,0.836071,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000


In [40]:
data.groupby('Sex')['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [41]:
data.groupby('SibSp')['Survived'].mean()

SibSp
0    0.345395
1    0.535885
2    0.464286
3    0.250000
4    0.166667
5    0.000000
8    0.000000
Name: Survived, dtype: float64

In [42]:
data.groupby('Parch')['Survived'].mean()

Parch
0    0.343658
1    0.550847
2    0.500000
3    0.600000
4    0.000000
5    0.200000
6    0.000000
Name: Survived, dtype: float64

In [43]:
data.pivot_table('Survived', index='Sex', columns='Pclass', aggfunc='mean')

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Missing and aberrant values

In [16]:
# Missing values 
(data.isnull()).sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Feature selection & Feature engineering 

In [46]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Modelisation 

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Array generation for training the model
X = pd.get_dummies(data[['Pclass', 'Sex']]).values # Transforms the categorical variables into integer values
X_test = pd.get_dummies(test_data[['Pclass', 'Sex']]).values
y = data['Survived'].values

# Random Forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X, y)

# Score 
clf.score(X, y)

# Predict 
predictions = clf.predict(X_test)

# Saving results 
results = pd.DataFrame({'PassengerId' : test_data.PassengerId, 
             'Survived' : predictions })
results.to_csv('Data/my_submission.csv', index=False)


## To do :

- Further exploration : correlation between variables / Volume of missing values / aberrant values 
- Feature selection (what feature I need to keep for my model ? )
- Feature engineering ( what feature I need to construct to improve my model ? )
- Model selection (Why this model ? What tests have been done ? )